In [1]:
# import the necessary packages
#from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import pickle
import time
import cv2
import os
from cv2_imshow import cv2_imshow
from align import AlignDlib

In [2]:
from model import create_model

Using TensorFlow backend.


In [3]:
print ('[INFO] loading face detector')
nn4_small_pretrained = create_model()
nn4_small_pretrained.load_weights('weights/nn4.small2.v1.h5')

print ('[INFO] loading emb face')
import pickle 
with open('database/x_vector.pkl', 'rb') as f:
    x_vector = pickle.load(f)
    
with open('database/x_label.pkl', 'rb') as f:
    x_label = pickle.load(f)
    
with open('database/x_name.pkl', 'rb') as f:
    x_name = pickle.load(f)

[INFO] loading face detector
[INFO] loading emb face


In [4]:
print (x_name)

['Donald_Rumsfeld', 'Donald_Rumsfeld', 'Donald_Rumsfeld', 'Donald_Rumsfeld', 'Donald_Rumsfeld', 'Donald_Rumsfeld', 'Donald_Rumsfeld', 'Donald_Rumsfeld', 'Donald_Rumsfeld', 'Donald_Rumsfeld', 'Arnold_Schwarzenegger', 'Arnold_Schwarzenegger', 'Arnold_Schwarzenegger', 'Arnold_Schwarzenegger', 'Arnold_Schwarzenegger', 'Arnold_Schwarzenegger', 'Arnold_Schwarzenegger', 'Arnold_Schwarzenegger', 'Arnold_Schwarzenegger', 'Arnold_Schwarzenegger', 'Ariel_Sharon', 'Ariel_Sharon', 'Ariel_Sharon', 'Ariel_Sharon', 'Ariel_Sharon', 'Ariel_Sharon', 'Ariel_Sharon', 'Ariel_Sharon', 'Ariel_Sharon', 'Ariel_Sharon', 'George_W_Bush', 'George_W_Bush', 'George_W_Bush', 'George_W_Bush', 'George_W_Bush', 'George_W_Bush', 'George_W_Bush', 'George_W_Bush', 'George_W_Bush', 'George_W_Bush', 'Jacques_Chirac', 'Jacques_Chirac', 'Jacques_Chirac', 'Jacques_Chirac', 'Jacques_Chirac', 'Jacques_Chirac', 'Jacques_Chirac', 'Jacques_Chirac', 'Jacques_Chirac', 'Jacques_Chirac', 'Tony_Blair', 'Tony_Blair', 'Tony_Blair', 'Tony_B

In [5]:
alignment = AlignDlib('models/landmarks.dat')
def align_image(img, box):
    return alignment.align(96, img, box,
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

def get_boxs(img):
    return alignment.getAllFaceBoundingBoxes(img)

def img2vect(img):
    img = (img / 255.).astype(np.float32)
    img = np.expand_dims(img, axis=0)
    return nn4_small_pretrained.predict(img)[0]

def draw(img, text, box):
    startX = box.left()
    startY = box.top()
    endX = startX + box.width()
    endY = startY + box.height()
    cv2.rectangle(img, (startX, startY), (endX, endY),(0, 0, 255), 2)
    cv2.putText(img, text, (startX, startY), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    return img

In [6]:
with open('models/svm.pkl', 'rb') as f:
    svm = pickle.load(f)

In [8]:
print ("[INFO] starting video stream...")
vs = cv2.VideoCapture(-1)
time.sleep(2.0)

# start the FPS throughput estimator
fps = FPS().start()

# loop over frames from the video file stream
while True:
    # grab the frame from the threaded video stream
    ret, frame = vs.read()
    
    boxs = get_boxs(frame)
       
    if boxs is not None: 
        for box in boxs:
            img = align_image(frame, box)
            yv = np.reshape(img2vect(img), (1, -1))
            
            dist = np.max(svm.decision_function(yv))
            print (svm.decision_function(yv))
            if dist < -0.2:
                print (dist)
                person = "unknow"
                print (person)
                print ('*'*60)
            else:
                lb = svm.predict(yv)
                for i, name in enumerate(x_name):
                    if x_label[i] == lb:
                        person = name
                        print (person)
                        print (dist)
                        print ('*'*60)
                
            draw(frame, person, box)
        
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    fps.update()
    if key == ord("q"):
        break
fps.stop()
cv2.destroyAllWindows()
vs.release()

[INFO] starting video stream...
[[-1.5524216  -0.93595962 -1.79483669 -2.65969418 -1.41399044 -2.06560915
  -0.4950049  -1.30719949 -1.29336765 -0.85400484 -0.46200901  0.32723323
  -1.20780992]]
Toan_Loi
0.32723322570167324
************************************************************
Toan_Loi
0.32723322570167324
************************************************************
Toan_Loi
0.32723322570167324
************************************************************
Toan_Loi
0.32723322570167324
************************************************************
Toan_Loi
0.32723322570167324
************************************************************
Toan_Loi
0.32723322570167324
************************************************************
Toan_Loi
0.32723322570167324
************************************************************
Toan_Loi
0.32723322570167324
************************************************************
Toan_Loi
0.32723322570167324
*******************************************************

[[-1.41853358 -1.06032407 -1.84859125 -2.57314429 -1.3131209  -2.00426633
  -0.73558735 -1.62588327 -1.28715058 -0.88586635 -0.43049441  0.42203804
  -1.04865855]]
Toan_Loi
0.42203804156947
************************************************************
Toan_Loi
0.42203804156947
************************************************************
Toan_Loi
0.42203804156947
************************************************************
Toan_Loi
0.42203804156947
************************************************************
Toan_Loi
0.42203804156947
************************************************************
Toan_Loi
0.42203804156947
************************************************************
Toan_Loi
0.42203804156947
************************************************************
Toan_Loi
0.42203804156947
************************************************************
Toan_Loi
0.42203804156947
************************************************************
Toan_Loi
0.42203804156947
***************************

[[-1.1650004  -1.1932866  -1.91866936 -2.18327091 -1.50977743 -2.13431918
  -0.94854493 -1.63071527 -1.17036035 -1.11281135 -0.18082614  0.27757407
  -1.20171473]]
Toan_Loi
0.2775740650443421
************************************************************
Toan_Loi
0.2775740650443421
************************************************************
Toan_Loi
0.2775740650443421
************************************************************
Toan_Loi
0.2775740650443421
************************************************************
Toan_Loi
0.2775740650443421
************************************************************
Toan_Loi
0.2775740650443421
************************************************************
Toan_Loi
0.2775740650443421
************************************************************
Toan_Loi
0.2775740650443421
************************************************************
Toan_Loi
0.2775740650443421
************************************************************
Toan_Loi
0.2775740650443421
*******

[[-1.35734946 -0.91532673 -1.76702575 -2.27149233 -1.48820994 -2.26001529
  -0.89675589 -1.52664652 -1.21198898 -1.16224652 -0.22168508  0.29228355
  -1.2141684 ]]
Toan_Loi
0.29228355191028976
************************************************************
Toan_Loi
0.29228355191028976
************************************************************
Toan_Loi
0.29228355191028976
************************************************************
Toan_Loi
0.29228355191028976
************************************************************
Toan_Loi
0.29228355191028976
************************************************************
Toan_Loi
0.29228355191028976
************************************************************
Toan_Loi
0.29228355191028976
************************************************************
Toan_Loi
0.29228355191028976
************************************************************
Toan_Loi
0.29228355191028976
************************************************************
Toan_Loi
0.292283551910289

[[-1.62445565 -0.68637692 -1.78597397 -2.6005636  -1.23811334 -2.21714893
  -1.02250569 -1.55298161 -1.16273627 -0.82714269 -0.55615501  0.38588999
  -1.10336942]]
Toan_Loi
0.3858899859930338
************************************************************
Toan_Loi
0.3858899859930338
************************************************************
Toan_Loi
0.3858899859930338
************************************************************
Toan_Loi
0.3858899859930338
************************************************************
Toan_Loi
0.3858899859930338
************************************************************
Toan_Loi
0.3858899859930338
************************************************************
Toan_Loi
0.3858899859930338
************************************************************
Toan_Loi
0.3858899859930338
************************************************************
Toan_Loi
0.3858899859930338
************************************************************
Toan_Loi
0.3858899859930338
*******

...

[[-1.51695822 -1.20697275 -1.9693355  -2.29839145 -1.16871878 -2.007588
  -1.09212467 -1.73495746 -1.10236645 -0.94285799 -0.20820938  0.2137931
  -1.07579301]]
Toan_Loi
0.2137931001915725
************************************************************
Toan_Loi
0.2137931001915725
************************************************************
Toan_Loi
0.2137931001915725
************************************************************
Toan_Loi
0.2137931001915725
************************************************************
Toan_Loi
0.2137931001915725
************************************************************
Toan_Loi
0.2137931001915725
************************************************************
Toan_Loi
0.2137931001915725
************************************************************
Toan_Loi
0.2137931001915725
************************************************************
Toan_Loi
0.2137931001915725
************************************************************
Toan_Loi
0.2137931001915725
**********

[[-1.63456815 -1.13898666 -1.43949315 -2.09148798 -0.88537021 -2.43597543
  -1.34752385 -1.82355928 -1.42230087 -1.05305362  0.17709257  0.07109642
  -0.7222433 ]]
Quang
0.17709256539317275
************************************************************
Quang
0.17709256539317275
************************************************************
Quang
0.17709256539317275
************************************************************
Quang
0.17709256539317275
************************************************************
Quang
0.17709256539317275
************************************************************
Quang
0.17709256539317275
************************************************************
Quang
0.17709256539317275
************************************************************
Quang
0.17709256539317275
************************************************************
Quang
0.17709256539317275
************************************************************
Quang
0.17709256539317275
***************************

[[-1.75860223 -1.63650852 -1.26655045 -2.28053053 -1.27287927 -2.0252708
  -1.44796021 -1.76278531 -1.17679769 -0.27992786 -0.37571725 -0.08673586
  -0.75512475]]
Toan_Loi
-0.08673585925605554
************************************************************
Toan_Loi
-0.08673585925605554
************************************************************
Toan_Loi
-0.08673585925605554
************************************************************
Toan_Loi
-0.08673585925605554
************************************************************
Toan_Loi
-0.08673585925605554
************************************************************
Toan_Loi
-0.08673585925605554
************************************************************
Toan_Loi
-0.08673585925605554
************************************************************
Toan_Loi
-0.08673585925605554
************************************************************
Toan_Loi
-0.08673585925605554
************************************************************
Toan_Loi
-0.086735

[[-1.71602557 -0.7791502  -1.46956823 -2.40803084 -1.32082806 -2.3674067
  -1.2904465  -1.60871297 -1.0407506  -0.68385292 -0.45711696  0.23092068
  -0.98262966]]
Toan_Loi
0.2309206750656747
************************************************************
Toan_Loi
0.2309206750656747
************************************************************
Toan_Loi
0.2309206750656747
************************************************************
Toan_Loi
0.2309206750656747
************************************************************
Toan_Loi
0.2309206750656747
************************************************************
Toan_Loi
0.2309206750656747
************************************************************
Toan_Loi
0.2309206750656747
************************************************************
Toan_Loi
0.2309206750656747
************************************************************
Toan_Loi
0.2309206750656747
************************************************************
Toan_Loi
0.2309206750656747
********

[[-1.93315316 -1.28988713 -1.50507522 -2.0569306  -0.98988412 -2.18501631
  -1.33249231 -2.01073466 -1.13734697 -0.75898163 -0.18293033 -0.01598606
  -0.84860045]]
Toan_Loi
-0.01598605837394551
************************************************************
Toan_Loi
-0.01598605837394551
************************************************************
Toan_Loi
-0.01598605837394551
************************************************************
Toan_Loi
-0.01598605837394551
************************************************************
Toan_Loi
-0.01598605837394551
************************************************************
Toan_Loi
-0.01598605837394551
************************************************************
Toan_Loi
-0.01598605837394551
************************************************************
Toan_Loi
-0.01598605837394551
************************************************************
Toan_Loi
-0.01598605837394551
************************************************************
Toan_Loi
-0.01598

In [8]:
vs.release()